In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow import keras 
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ConvLSTM2D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import pandas as pd
import glob
tf.config.optimizer.set_jit(True)
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
def files_to_score_df(path):
    files = [f for f in glob.glob(path + "*")]
    scores = []
    for i in files:
        scores.append(float(i[5:8]))
    return pd.DataFrame(files,scores).reset_index()

In [4]:
score_df = files_to_score_df('data/')

In [5]:
score_df

,index,0
0,9.1,data\9.1-SLSSC2019.avi
1,9.1,data\9.1-SLSSC2019.mov


In [25]:
score_df.iloc[1][0]

'data\\9.1-SLSSC2019.mov'

In [9]:
def frames_extraction(video_path):
    # Empty List declared to store video frames
    frames_list = []
    
    # Reading the Video File Using the VideoCapture
    video_reader = cv2.VideoCapture(video_path)

    # Iterating through Video Frames
    while True:

        # Reading a frame from the video file 
        success, frame = video_reader.read() 
    
        # If Video frame was not successfully read then break the loop
        if not success:
            break
            
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = gray / 255
        
        # Appending the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    # Closing the VideoCapture object and releasing all resources. 
    video_reader.release()

    # returning the frames list 
    return frames_list


In [10]:
test_df = pd.DataFrame([[frames_extraction(score_df.iloc[1][0])]], np.array([score_df.iloc[1]['index']])).reset_index()

In [35]:
X = test_df.iloc[0][0]
y = test_df.iloc[0]['index']

In [36]:
X = X[1:]

In [43]:
X = [X]

In [44]:
X = np.array(X)

In [40]:
X

array([[[[0.0745098 , 0.07058824, 0.06666667, ..., 0.18431373,
          0.18431373, 0.18823529],
         [0.09411765, 0.09803922, 0.09803922, ..., 0.1372549 ,
          0.1372549 , 0.1372549 ],
         [0.0627451 , 0.06666667, 0.06666667, ..., 0.19607843,
          0.2       , 0.19607843],
         ...,
         [0.15294118, 0.15294118, 0.15294118, ..., 0.7372549 ,
          0.7372549 , 0.7372549 ],
         [0.15294118, 0.15294118, 0.15294118, ..., 0.7372549 ,
          0.7372549 , 0.7372549 ],
         [0.15294118, 0.15294118, 0.15294118, ..., 0.7372549 ,
          0.7372549 , 0.7372549 ]],

        [[0.08627451, 0.08627451, 0.08627451, ..., 0.12156863,
          0.12156863, 0.11764706],
         [0.07058824, 0.07058824, 0.06666667, ..., 0.19215686,
          0.19215686, 0.19215686],
         [0.07843137, 0.0745098 , 0.0745098 , ..., 0.1254902 ,
          0.12156863, 0.11764706],
         ...,
         [0.15294118, 0.15294118, 0.15294118, ..., 0.7372549 ,
          0.7372549 , 0.7

In [41]:
y = [y]
y = np.array(y)

In [28]:
y

array([9.1])

In [29]:
X.shape

(1, 61, 800, 800)

In [58]:
model = Sequential()
model.add(ConvLSTM2D(60, kernel_size=(2,2), input_shape=(1, 60, 800, 800)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=50)

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 81.7405
Epoch 2/50
1/1 [==============================] - 1s 1s/step - loss: 50.3697
Epoch 3/50
1/1 [==============================] - 1s 1s/step - loss: 36.3254
Epoch 4/50
1/1 [==============================] - 1s 1s/step - loss: 27.5382
Epoch 5/50
1/1 [==============================] - 1s 1s/step - loss: 20.0899
Epoch 6/50
1/1 [==============================] - 1s 1s/step - loss: 14.7081
Epoch 7/50
1/1 [==============================] - 1s 1s/step - loss: 11.1694
Epoch 8/50
1/1 [==============================] - 1s 1s/step - loss: 8.6507
Epoch 9/50
1/1 [==============================] - 1s 1s/step - loss: 6.7367
Epoch 10/50
1/1 [==============================] - 1s 1s/step - loss: 5.4430
Epoch 11/50
1/1 [==============================] - 1s 1s/step - loss: 4.5717
Epoch 12/50
1/1 [==============================] - 1s 1s/step - loss: 3.8210
Epoch 13/50
1/1 [==============================] - 1s 1s/step - loss: 3.1611
E

In [60]:
results = model.predict(X)

In [66]:
len(results[0][0])

799

# Grayscale Code

In [7]:
source = cv2.VideoCapture(score_df.iloc[0][0])
# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(source.get(3)) 
frame_height = int(source.get(4)) 
   
size = (frame_width, frame_height) 

result = cv2.VideoWriter('gray.avi',  
            cv2.VideoWriter_fourcc(*'MJPG'), 
            30, size, 0) 
  
# running the loop 
while True: 
  
    # extracting the frames 
    ret, img = source.read() 
      
    # converting to gray-scale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    # write to gray-scale 
    result.write(gray)

    # displaying the video 
    cv2.imshow("Live", gray) 
  
    # exiting the loop 
    key = cv2.waitKey(1) 
    if key == ord("q"): 
        break
# closing the window 
cv2.destroyAllWindows() 
source.release()

error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
